<a href="https://colab.research.google.com/github/stefandanielachirei/licenta-2025-alexandra-florea/blob/main/Research/DistilBERT/Recente/softwareRequirementsExtendedLicenta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning Transformer for MultiClass Text Classification

### Introduction

In this tutorial we will be fine tuning a transformer model for the **Multiclass text classification** problem.
This is one of the most common business problems where a given piece of text/sentence/document needs to be classified into one of the categories out of the given list.

#### Flow of the notebook

The notebook will be divided into seperate sections to provide a organized walk through for the process used. This process can be modified for individual use cases. The sections are:

1. [Importing Python Libraries and preparing the environment](#section01)
2. [Importing and Pre-Processing the domain data](#section02)
3. [Preparing the Dataset and Dataloader](#section03)
4. [Creating the Neural Network for Fine Tuning](#section04)
5. [Fine Tuning the Model](#section05)
6. [Validating the Model Performance](#section06)
7. [Saving the model and artifacts for Inference in Future](#section07)

#### Technical Details

This script leverages on multiple tools designed by other teams. Details of the tools used below. Please ensure that these elements are present in your setup to successfully implement this script.

 - Data:
	 - We are using the News aggregator dataset available at by [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/News+Aggregator)
	 - We are referring only to the first csv file from the data dump: `newsCorpora.csv`
	 - There are `422937` rows of data.  Where each row has the following data-point:
		 - ID Numeric ID  
		 - TITLE News title  
		 - URL Url  
		 - PUBLISHER Publisher name  
		 - CATEGORY News category (b = business, t = science and technology, e = entertainment, m = health)  
		 - STORY Alphanumeric ID of the cluster that includes news about the same story  
		 - HOSTNAME Url hostname  
		 - TIMESTAMP Approximate time the news was published, as the number of milliseconds since the epoch 00:00:00 GMT, January 1, 1970


 - Language Model Used:
	 - DistilBERT this is a smaller transformer model as compared to BERT or Roberta. It is created by process of distillation applied to Bert.
	 - [Blog-Post](https://medium.com/huggingface/distilbert-8cf3380435b5)
	 - [Research Paper](https://arxiv.org/abs/1910.01108)
     - [Documentation for python](https://huggingface.co/transformers/model_doc/distilbert.html)


 - Hardware Requirements:
	 - Python 3.6 and above
	 - Pytorch, Transformers and All the stock Python ML Libraries
	 - GPU enabled setup


 - Script Objective:
	 - The objective of this script is to fine tune DistilBERT to be able to classify a news headline into the following categories:
		 - Business
		 - Technology
		 - Health
		 - Entertainment


In [ ]:
!pip install tensorflow-cpu
!pip install -q tensorboard

<a id='section01'></a>
### Importing Python Libraries and preparing the environment

At this step we will be importing the libraries and modules needed to run our script. Libraries are:
* Pandas
* Pytorch
* Pytorch Utils for Dataset and Dataloader
* Transformers
* DistilBERT Model and Tokenizer

Followed by that we will preapre the device for CUDA execeution. This configuration is needed if you want to leverage on onboard GPU.

In [ ]:
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

In [ ]:
import tensorflow as tf
import datetime
import os

In [ ]:
log_dir = "/content/drive/MyDrive/Licenta/MyLogs/Software_requirements_extended/" + datetime.datetime.now().strftime("%d-%m-%Y-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

<a id='section02'></a>
### Importing and Pre-Processing the domain data

We will be working with the data and preparing for fine tuning purposes.
*Assuming that the `newCorpora.csv` is already downloaded in your `data` folder*

Import the file in a dataframe and give it the headers as per the documentation.
Cleaning the file to remove the unwanted columns and create an additional column for training.
The final Dataframe will be something like this:

|TITLE|CATEGORY|ENCODED_CAT|
|--|--|--|
|  title_1|Entertainment | 1 |
|  title_2|Entertainment | 1 |
|  title_3|Business| 2 |
|  title_4|Science| 3 |
|  title_5|Science| 3 |
|  title_6|Health| 4 |

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Licenta/data/software_requirements_extended.csv', sep='\t', names=['TYPE','REQUIREMENTS'], on_bad_lines='skip', encoding='latin-1')
# df.head()
#df = df[['TITLE','CATEGORY']]
# df.head()

my_dict = {
    'A':'Availability',
    'F':'Functional',
    'FR':'Functional Requirement',
    'FT':'Fault Tolerance',
    'L':'Legal',
    'LF':'Look and Feel',
    'MN':'Maintainability',
    'O':'Operational',
    'PE':'Performance',
    'PO':'Portability',
    'SC':'Scalability',
    'SE':'Security',
    'US':'Usability',
    'NFR':'Non-Functional Requirement'
}

def update_cat(x):
     return my_dict.get(x, x)

df['TYPE'] = df['TYPE'].apply(lambda x: update_cat(x))

encode_dict = {}

def encode_cat(x):
    if x not in encode_dict.keys():
        encode_dict[x]=len(encode_dict)
    return encode_dict[x]

df['ENCODE_CAT'] = df['TYPE'].apply(lambda x: encode_cat(x))

<a id='section03'></a>
### Preparing the Dataset and Dataloader

We will start with defining few key variables that will be used later during the training/fine tuning stage.
Followed by creation of Dataset class - This defines how the text is pre-processed before sending it to the neural network. We will also define the Dataloader that will feed  the data in batches to the neural network for suitable training and processing.
Dataset and Dataloader are constructs of the PyTorch library for defining and controlling the data pre-processing and its passage to neural network. For further reading into Dataset and Dataloader read the [docs at PyTorch](https://pytorch.org/docs/stable/data.html)

#### *Triage* Dataset Class
- This class is defined to accept the Dataframe as input and generate tokenized output that is used by the DistilBERT model for training.
- We are using the DistilBERT tokenizer to tokenize the data in the `TITLE` column of the dataframe.
- The tokenizer uses the `encode_plus` method to perform tokenization and generate the necessary outputs, namely: `ids`, `attention_mask`
- To read further into the tokenizer, [refer to this document](https://huggingface.co/transformers/model_doc/distilbert.html#distilberttokenizer)
- `target` is the encoded category on the news headline.
- The *Triage* class is used to create 2 datasets, for training and for validation.
- *Training Dataset* is used to fine tune the model: **80% of the original data**
- *Validation Dataset* is used to evaluate the performance of the model. The model has not seen this data during training.

#### Dataloader
- Dataloader is used to for creating training and validation dataloader that load data to the neural network in a defined manner. This is needed because all the data from the dataset cannot be loaded to the memory at once, hence the amount of dataloaded to the memory and then passed to the neural network needs to be controlled.
- This control is achieved using the parameters such as `batch_size` and `max_len`.
- Training and Validation dataloaders are used in the training and validation part of the flow respectively

In [ ]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 30
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

In [ ]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        text = str(self.data.TYPE[index]) + " " + str(self.data.REQUIREMENTS[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        ids = inputs['input_ids'].squeeze(0)
        mask = inputs['attention_mask'].squeeze(0)


        return {
            'ids': ids,
            'mask': mask,
            'targets': torch.tensor(self.data.ENCODE_CAT[index], dtype=torch.long)
        }

    def __len__(self):
        return self.len

In [ ]:

train_size = 0.6
validation_size = 0.2


train_dataset=df.sample(frac=train_size,random_state=200)
remaining_dataset=df.drop(train_dataset.index).reset_index(drop=True)

validation_dataset = remaining_dataset.sample(frac=validation_size/(1-train_size), random_state=200)
test_dataset = remaining_dataset.drop(validation_dataset.index).reset_index(drop=True)

train_dataset = train_dataset.reset_index(drop=True)
validation_dataset = validation_dataset.reset_index(drop=True)
test_dataset = test_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))
print("VALIDATION Dataset: {}".format(validation_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
validation_set = Triage(validation_dataset, tokenizer, MAX_LEN)
testing_set = Triage(test_dataset, tokenizer, MAX_LEN)

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

validation_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)
validation_loader = DataLoader(validation_set, **validation_params)

<a id='section04'></a>
### Creating the Neural Network for Fine Tuning

#### Neural Network
 - We will be creating a neural network with the `DistillBERTClass`.
 - This network will have the DistilBERT Language model followed by a `dropout` and finally a `Linear` layer to obtain the final outputs.
 - The data will be fed to the DistilBERT Language model as defined in the dataset.
 - Final layer outputs is what will be compared to the `encoded category` to determine the accuracy of models prediction.
 - We will initiate an instance of the network called `model`. This instance will be used for training and then to save the final trained model for future inference.

#### Loss Function and Optimizer
 - `Loss Function` and `Optimizer` and defined in the next cell.
 - The `Loss Function` is used the calculate the difference in the output created by the model and the actual output.
 - `Optimizer` is used to update the weights of the neural network to improve its performance.

#### Further Reading
- You can refer to my [Pytorch Tutorials](https://github.com/abhimishra91/pytorch-tutorials) to get an intuition of Loss Function and Optimizer.
- [Pytorch Documentation for Loss Function](https://pytorch.org/docs/stable/nn.html#loss-functions)
- [Pytorch Documentation for Optimizer](https://pytorch.org/docs/stable/optim.html)
- Refer to the links provided on the top of the notebook to read more about DistiBERT.

In [ ]:

class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 14)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = DistillBERTClass()
model.to(device)

In [ ]:

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

<a id='section05'></a>
### Fine Tuning the Model

After all the effort of loading and preparing the data and datasets, creating the model and defining its loss and optimizer. This is probably the easier steps in the process.

Here we define a training function that trains the model on the training dataset created above, specified number of times (EPOCH), An epoch defines how many times the complete data will be passed through the network.

Following events happen in this function to fine tune the neural network:
- The dataloader passes data to the model based on the batch size.
- Subsequent output from the model and the actual category are compared to calculate the loss.
- Loss value is used to optimize the weights of the neurons in the network.
- After every 5000 steps the loss value is printed in the console.

As you can see just in 1 epoch by the final step the model was working with a miniscule loss of 0.0002485 i.e. the output is extremely close to the actual output.

In [ ]:

def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np

writer = SummaryWriter(log_dir=f"/content/drive/MyDrive/Licenta/MyLogs/Software_requirements_extended/{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}")


def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0

    model.train()
    for step, data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        targets = data['targets'].to(device, dtype=torch.long)

        targets = targets % model.classifier.out_features
        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        writer.add_scalar("Training Loss", loss.item(), epoch * len(training_loader) + step)
        writer.add_scalar("Training Accuracy", (n_correct * 100) / nb_tr_examples, epoch * len(training_loader) + step)

        if step % 5000 == 0:
            loss_step = tr_loss / nb_tr_steps
            accu_step = (n_correct * 100) / nb_tr_examples
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'\nThe Total Accuracy for Epoch {epoch}: {(n_correct * 100) / nb_tr_examples}\n')
    epoch_loss = tr_loss / nb_tr_steps
    epoch_accu = (n_correct * 100) / nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    writer.add_scalar("Training Loss per Epoch", epoch_loss, epoch)
    writer.add_scalar("Training Accuracy per Epoch", epoch_accu, epoch)

    return


In [ ]:

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = np.inf
        self.early_stop = False

    def __call__(self, val_loss):
        if (self.best_loss - val_loss) > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            print(f"EarlyStopping: {self.counter}/{self.patience} epoci fără îmbunătățire.")
            if self.counter >= self.patience:
                self.early_stop = True


def validate():
    val_loss = 0
    model.eval()
    with torch.no_grad():
        for step, data in enumerate(validation_loader, 0):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.long)

            targets = targets % model.classifier.out_features
            outputs = model(ids, mask)
            loss = loss_function(outputs, targets)

            val_loss += loss.item()

    val_loss /= len(validation_loader)

    scheduler.step(val_loss)
    return val_loss

early_stopping = EarlyStopping(patience=5, min_delta=0.01)
validation_losses = []

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)

for epoch in range(EPOCHS):
    train(epoch)

    val_loss = validate()
    validation_losses.append(val_loss)
    print(f"Validation Loss Epoch {epoch}: {val_loss}")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print(f"Early stopping la epoca {epoch}. Performanța pe setul de validare nu se mai îmbunătățește.")
        break
    print("-----------------------------------------------------------------------")

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/Licenta/MyLogs/Software_requirements_extended
#%reload_ext tensorboard

<a id='section06'></a>
### Validating the Model

During the validation stage we pass the unseen data(Testing Dataset) to the model. This step determines how good the model performs on the unseen data.

This unseen data is the 20% of `newscorpora.csv` which was seperated during the Dataset creation stage.
During the validation stage the weights of the model are not updated. Only the final output is compared to the actual value. This comparison is then used to calcuate the accuracy of the model.

As you can see the model is predicting the correct category of a given headline to a 99.9% accuracy.

In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss = 0; nb_tr_steps = 0; nb_tr_examples = 0 # Initialize variables
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)


            targets = targets % model.classifier.out_features

            outputs = model(ids, mask)
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)

            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    return epoch_accu

In [ ]:
print('This is the validation section to print the accuracy and see how it performs')
print('Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch')

acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

<a id='section07'></a>
### Saving the Trained Model Artifacts for inference

This is the final step in the process of fine tuning the model.

The model and its vocabulary are saved locally. These files are then used in the future to make inference on new inputs of news headlines.

Please remember that a trained neural network is only useful when used in actual inference after its training.

In the lifecycle of an ML projects this is only half the job done. We will leave the inference of these models for some other day.

In [ ]:
import torch
from transformers import DistilBertTokenizerFast

save_directory = '/content/drive/MyDrive/Licenta/models/Modele_salvate/Software_requirements_extended/'

tokenizer.save_pretrained(save_directory)

torch.save(model.state_dict(), save_directory + 'pytorch_model.bin')

print("Modelul și tokenizerul au fost salvați cu succes!")


In [ ]:
import torch
from transformers import DistilBertTokenizerFast

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

load_directory = '/content/drive/MyDrive/Licenta/models/Modele_salvate/Software_requirements_extended/'

model = DistillBERTClass()
model.load_state_dict(torch.load(load_directory + 'pytorch_model.bin', map_location=device))
model.to(device)
model.eval()

tokenizer = DistilBertTokenizerFast.from_pretrained(load_directory)

print("Modelul și tokenizerul au fost încărcate cu succes!")


In [ ]:
texts = [
    "The system must ensure high availability and minimal downtime.",
    "The software shall provide core functional capabilities as specified.",
    "Detailed functional requirements must be clearly defined and met.",
    "The system should tolerate faults and continue operation during errors.",
    "All operations must comply with relevant legal regulations.",
    "The user interface should be visually appealing and intuitive.",
    "The software must be easy to maintain and update.",
    "Operational constraints must be considered during deployment.",
    "The system should meet performance targets for speed and efficiency.",
    "The application must be portable across multiple platforms.",
    "The system needs to scale efficiently as user demand grows.",
    "Security features such as authentication and data encryption are required.",
    "The system should be easy and pleasant to use for end-users.",
    "Non-functional requirements such as reliability and usability must be satisfied."
]

inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
inputs = {k: v.to(device) for k, v in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)
    preds = outputs.argmax(dim=1).tolist()

encode_dict = {
    'Availability': 0,
    'Functional': 1,
    'Functional Requirement': 2,
    'Fault Tolerance': 3,
    'Legal': 4,
    'Look and Feel': 5,
    'Maintainability': 6,
    'Operational': 7,
    'Performance': 8,
    'Portability': 9,
    'Scalability': 10,
    'Security': 11,
    'Usability': 12,
    'Non-Functional Requirement': 13
}

label_names = [None] * len(encode_dict)
for name, idx in encode_dict.items():
    label_names[idx] = name

for text, pred in zip(texts, preds):
    print(f"{text} → {label_names[pred]}")
